In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import regex as re
import seaborn as sns
from collections import Counter

In [2]:
df_master = pd.read_excel(r"data/SuperRare_master.xlsx")
df_bids = pd.read_csv(r"C:\Users\culm-erbach\Documents\VW NLP\requirements\data\bids.csv")

### Create min and max from bids and merge them to the Master dataframe

In [3]:
df_bids_min = df_bids.groupby('tokenId', as_index=False)['usd'].min()
df_bids_min.rename(columns={'usd': 'usd_min_bids'}, inplace=True)
df_bids = df_bids.merge(df_bids_min, left_on="tokenId", right_on="tokenId")

In [4]:
df_bids_max = df_bids.groupby('tokenId', as_index=False)['usd'].max()
df_bids_max.rename(columns={'usd': 'usd_max_bids'}, inplace=True)
df_bids = df_bids.merge(df_bids_max, left_on="tokenId", right_on="tokenId")

In [5]:
df = df_master.merge(df_bids, left_on="tokenId", right_on="tokenId")

## Create Dummies for words of Kategories

In [6]:
keywords = ['Tweet', 'Twitter', 'Instagram', 'Facebook', 'snapchat', 'posts', 'link', 'socialmedia', 'tag', 'emoji', 'hashtag', 'socail media', 'Portrait', 'Painting', 'contemporary, modern', 'acrylic', 'surrealism', 'minimalism', 'fine art', 'ink', 'painted', 'pencil', 'drawing, expressionism', 'dada', 
                'constructivism', 'futurism', 'cubism', 'impressionism', 'realism', 'Baroque', 'Banksy', 'Graffiti', 'pop art', 'street art', ' old masters', 'abstract', 'Meme', 'gaming', 'comics', 'software', 'gamers', 'mythology', 'gods', 'magic', 'mystic', 'cat', 'dog', 'hero', 'assassin', 'GIF', 'Pixel', '3d', 'animation', 
           'photoshop', 'video', 'image', 'png', 'jpeg', 'mp4', 'iPhone', 'smartphone', 'photography', 'venus', 'Athena', 'TV-series', 'comics', 'phantasy', 'saint', 'galaxy', 'Fire', 'dark', 'dead', 'red', 'murder', 'dystopia', 'broken', 'shadows', 'darkness', 'blood', 'alien', 'violence', 'weapons', 'guns', 'RIP', 'dystopia', 
                'depression', 'anxiety', 'Crypto', 'Blockchain', 'Ethereum', 'Bitcoin', 'ETH', 'digital', 'quantum', 'cryptocurrency', 'digitally', 'cyberspace', 'collage', 'mining', 'NFT', 
          'money', 'tokenized', 'coins', 'token', 'XCopy', 'collage', 'algorithm', 'generate', 'process', 'artificial intelligence', 'machine', 'data science', 'natural science', 'physics', 'computer science', 
      'robot', 'virtual reality', 'data',  'matrix','Nature', 'waterfalls', 'sky', 'beach', 'travel', 'day', 'night', 'island', 'Venice', 'Los Angeles', 'natural', 'sunset', 'snow', 'ocean', 'city', 'season', 'roses', 
          'climate', 'flower', 'adventures', 'rainbow', 'wood']

In [7]:
df['keywords'] = df.namedescriptiontags.str.extract('({0})'.format('|'.join(keywords)), flags=re.IGNORECASE)

In [8]:
df = pd.get_dummies(df, prefix='keywords_', columns=['keywords'])

## Extract most common words

In [9]:
stop_words =  ['the', 'a ','us', 'of', 'in', 'that', 'made', 'we\'ve', 'after','their','his','only','previously', 'by', 'this', 'with', 'a', 'but', 'it', 'or', 'i', 'my', 'be', 'to', 'we', 'you', 'them', 'they', 'at', 'and',
             'is', 'on', 'for', 'an', 'was', 'are', 'have', 'will', 'not', 'can', 'as', 'what', '.', ',', '-', '/', '!', '?', '()', '(', ')', '+', '#', '*', '&', '//', 'has', 'me', 'so', 'our', 'when', 'its', 'all', 'from', 'your', 'into', 'who']

words_new = (df['namedescriptiontags']
           .str.lower()
           .str.cat(sep=' ')
           .split()
)

common_words = pd.DataFrame(Counter(words_new).most_common(100),
                    columns=['Word', 'Frequency'])

list_words = common_words['Word']
list_words = list_words.to_list()
list_common_words = [i for i in list_words if i not in stop_words]
list_common_words

['3d',
 'art',
 'gif',
 'digital',
 'animation',
 'abstract',
 'surreal',
 'illustration',
 'cryptoart',
 'portrait',
 'painting',
 'one',
 'animated',
 'loop',
 'artwork',
 '3dart',
 'x',
 'digitalart',
 'created',
 'bitcoin',
 'generative',
 'crypto',
 'piece',
 'nft',
 'ai',
 'series',
 'new',
 'which',
 'psychedelic',
 'time',
 'more',
 'drawing',
 'color',
 'superrare',
 'rare',
 'first',
 'surrealism',
 'world',
 'ethereum',
 'life',
 'work',
 'love',
 'glitch',
 'like',
 'original',
 'space',
 'blockchain',
 'nature',
 'no',
 'through',
 'gan']

### Create Dummies for most common words

In [10]:
df['common_words'] = df.namedescriptiontags.str.extract('({0})'.format('|'.join(keywords)), flags=re.IGNORECASE)

In [11]:
df = pd.get_dummies(df, prefix='common_', columns=['common_words'])

In [12]:
df

,Unnamed: 0,timestamp_x,tokenId,buyer,seller,eth_x,rate_x,usd_x,contract_x,transactionId_x,namedescriptiontags,description_category,image,media,type,size,dimensions,creator,owner,timestamp_y,contract_y,transactionId_y,tokenId_count_bids,usd_mean_bids,tokenId.1,timestamp_x.1,Covid_dummy,keyword_social_media,keyword_art_movement,keyword_gaming,keyword_dark,keyword_crypto,keyword_AI,keyword_nature,keyowrd_other,2018_Q2,2018_Q3,2018_Q4,2019_Q1,2019_Q2,2019_Q3,2019_Q4,2020_Q1,2020_Q2,2020_Q3,2020_Q4,2021_Q1,dollar,timestamp,bidder,eth_y,rate_y,usd_y,contract,transactionId,usd_min_bids,usd_max_bids,keywords__3D,keywords__3d,keywords__ABSTRACT,keywords__ALGORITHM,keywords__ALIEN,keywords__ANIMATION,keywords__ANXIETY,keywords__ANimation,keywords__ATHENA,keywords__Abstract,keywords__Acrylic,keywords__Adventures,keywords__Algorithm,keywords__Alien,keywords__Animation,keywords__Anxiety,keywords__Artificial Intelligence,keywords__Artificial intelligence,keywords__Assassin,keywords__Athena,keywords__BAROQUE,keywords__BEACH,keywords__BITCOIN,keywords__BROKEN,keywords__Banksy,keywords__Baroque,keywords__Beach,keywords__BitCoin,keywords__Bitcoin,keywords__BlockChain,keywords__Blockchain,keywords__Blood,keywords__Broken,keywords__CAT,keywords__CITY,keywords__CLIMATE,keywords__COLLAGE,keywords__CRYPTO,keywords__Cat,keywords__City,keywords__Climate,keywords__Coins,keywords__Collage,keywords__Comics,keywords__Crypto,keywords__Cubism,keywords__Cyberspace,keywords__DADA,keywords__DARK,keywords__DAY,keywords__DEAD,keywords__DIGITAL,keywords__DIgital,keywords__DOG,keywords__Dark,keywords__Data,keywords__Day,keywords__Dead,keywords__Depression,keywords__Digital,keywords__Dog,keywords__Dystopia,keywords__EMOJI,keywords__ETH,keywords__ETHEREUM,keywords__Emoji,keywords__Eth,keywords__Ethereum,keywords__FACEBOOK,keywords__FIRE,keywords__FLOWER,keywords__Facebook,keywords__Fine Art,keywords__Fire,keywords__Flower,keywords__GENERATE,keywords__GIF,keywords__GIf,keywords__GRAFFITI,keywords__Galaxy,keywords__Gaming,keywords__Generate,keywords__Gif,keywords__Gods,keywords__Graffiti,keywords__Guns,keywords__HERO,keywords__Hashtag,keywords__Hero,keywords__IMAGE,keywords__INK,keywords__Image,keywords__Ink,keywords__Instagram,keywords__Island,keywords__JPEG,keywords__Jpeg,keywords__Link,keywords__Los Angeles,keywords__MACHINE,keywords__MATRIX,keywords__MEME,keywords__MONEY,keywords__MP4,keywords__Machine,keywords__Magic,keywords__Matrix,keywords__Meme,keywords__Minimalism,keywords__Mining,keywords__Money,keywords__Mp4,keywords__Mystic,keywords__Mythology,keywords__NATURE,keywords__NFT,keywords__NIGHT,keywords__Natural,keywords__Nature,keywords__Night,keywords__Ocean,keywords__PAINTING,keywords__PIXEL,keywords__PNG,keywords__PORTRAIT,keywords__Painted,keywords__Painting,keywords__Photography,keywords__Photoshop,keywords__Physics,keywords__Pixel,keywords__Pop Art,keywords__Portrait,keywords__Process,keywords__QUANTUM,keywords__Quantum,keywords__RED,keywords__RIP,keywords__ROBOT,keywords__Rainbow,keywords__Realism,keywords__Red,keywords__Rip,keywords__RoSeS,keywords__Robot,keywords__Roses,keywords__SAINT,keywords__SMARTPHONE,keywords__SURREALISM,keywords__Saint,keywords__Season,keywords__Shadows,keywords__Sky,keywords__Snow,keywords__Software,keywords__Street Art,keywords__Sunset,keywords__Surrealism,keywords__TAG,keywords__TOKEN,keywords__Tag,keywords__Token,keywords__Tokenized,keywords__Travel,keywords__Twitter,keywords__VENUS,keywords__VIDEO,keywords__Venice,keywords__Venus,keywords__Video,keywords__Violence,keywords__Virtual Reality,keywords__WOOD,keywords__Waterfalls,keywords__Weapons,keywords__Wood,keywords__XCOPY,keywords__abstract,keywords__acrylic,keywords__adventures,keywords__algorithm,keywords__alien,keywords__animation,keywords__anxiety,keywords__artificial intelligence,keywords__banksy,keywords__baroque,keywords__beach,keywords__bitcoin,keywords__blockchain,keywords__blood,keywords__broken,keywords__cat,keywords__city,keywords__climate,keywords__coins,ke